In [23]:
import pandas as pd
df = pd.read_csv("Results_21Mar2022.csv")

mean_cols = [col for col in df.columns if col.startswith("mean_")]
group_cols = ["diet_group", "sex", "age_group"]
all_cols_to_keep = group_cols + mean_cols + ["n_participants"]

agg_dict = {col: "mean" for col in mean_cols}
agg_dict["n_participants"] = "first"

df_grouped = df[all_cols_to_keep].groupby(group_cols).agg(agg_dict).reset_index()
df_grouped.head()

,diet_group,sex,age_group,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,mean_acid,n_participants
0,fish,female,20-29,4.902947,6.489031,17131.963352,20.985464,0.842864,0.322486,248.135212,759.237211,22.770956,1329
1,fish,female,30-39,4.942664,6.515886,17652.628540,20.948442,0.836500,0.321971,249.827684,759.629613,23.257771,2024
2,fish,female,40-49,4.927040,6.540682,17729.964756,20.713798,0.841106,0.320413,252.846922,748.488755,23.470170,1689
3,fish,female,50-59,4.944320,6.566172,18446.193259,20.989284,0.834881,0.323892,246.789912,788.932276,23.652474,946
4,fish,female,60-69,5.103823,6.866635,18922.366831,21.569322,0.879670,0.336023,258.645759,819.856604,24.455240,492


In [25]:
selected_cols = ["mean_ghgs", "mean_ghgs_ch4", "mean_ghgs_n2o", "mean_acid", "mean_eut"]
df_norm = df_grouped[selected_cols].copy()

#Min-Max Normalisation + Reverse Processing
for col in selected_cols:
    min_val = df_norm[col].min()
    max_val = df_norm[col].max()
    df_norm[col] = (max_val - df_norm[col]) / (max_val - min_val)

df_norm.head()

,mean_ghgs,mean_ghgs_ch4,mean_ghgs_n2o,mean_acid,mean_eut
0,0.744986,0.713425,0.827469,0.705237,0.737148
1,0.740967,0.715909,0.828186,0.693554,0.738089
2,0.742548,0.714111,0.830353,0.688456,0.744056
3,0.740799,0.716541,0.825513,0.684081,0.737051
4,0.724656,0.699058,0.808637,0.664815,0.722302


In [27]:
selected_cols = ["mean_ghgs", "mean_ghgs_ch4", "mean_ghgs_n2o", "mean_acid", "mean_eut"]
norm_cols = [col + "_norm" for col in selected_cols]

for col in norm_cols:
    if col in df_grouped.columns:
        df_grouped.drop(columns=col, inplace=True)

df_norm = pd.DataFrame()
for col in selected_cols:
    min_val = df_grouped[col].min()
    max_val = df_grouped[col].max()
    df_norm[col + "_norm"] = (max_val - df_grouped[col]) / (max_val - min_val)

df_grouped = pd.concat([df_grouped, df_norm], axis=1)

df_grouped.head()

,diet_group,sex,age_group,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,mean_acid,n_participants,mean_ghgs_norm,mean_ghgs_ch4_norm,mean_ghgs_n2o_norm,mean_acid_norm,mean_eut_norm
0,fish,female,20-29,4.902947,6.489031,17131.963352,20.985464,0.842864,0.322486,248.135212,759.237211,22.770956,1329,0.744986,0.713425,0.827469,0.705237,0.737148
1,fish,female,30-39,4.942664,6.515886,17652.628540,20.948442,0.836500,0.321971,249.827684,759.629613,23.257771,2024,0.740967,0.715909,0.828186,0.693554,0.738089
2,fish,female,40-49,4.927040,6.540682,17729.964756,20.713798,0.841106,0.320413,252.846922,748.488755,23.470170,1689,0.742548,0.714111,0.830353,0.688456,0.744056
3,fish,female,50-59,4.944320,6.566172,18446.193259,20.989284,0.834881,0.323892,246.789912,788.932276,23.652474,946,0.740799,0.716541,0.825513,0.684081,0.737051
4,fish,female,60-69,5.103823,6.866635,18922.366831,21.569322,0.879670,0.336023,258.645759,819.856604,24.455240,492,0.724656,0.699058,0.808637,0.664815,0.722302


In [29]:
import numpy as np

norm_cols = [col + "_norm" for col in ["mean_ghgs", "mean_ghgs_ch4", "mean_ghgs_n2o", "mean_acid", "mean_eut"]]
X = df_grouped[norm_cols].values

#Calculate the weight of each indicator using the entropy weight method
col_sum = X.sum(axis=0)

pij = X / col_sum
# Avoid log(0) errors: replace 0 with a very small value.
pij = np.where(pij == 0, 1e-12, pij)
n = X.shape[0]
k = 1 / np.log(n)
#Calculate the entropy value ej
ej = -k * (pij * np.log(pij)).sum(axis=0)
#Coefficient of variation dj & weights wj
dj = 1 - ej
weights = dj / dj.sum()


for col, weight in zip(norm_cols, weights):
    print(f"{col}: Weight = {weight:.4f}")

mean_ghgs_norm: Weight = 0.2297
mean_ghgs_ch4_norm: Weight = 0.2303
mean_ghgs_n2o_norm: Weight = 0.1963
mean_acid_norm: Weight = 0.1958
mean_eut_norm: Weight = 0.1479


In [31]:
# Calculate weighted scores 
df_grouped["eco_score"] = df_grouped[norm_cols].values.dot(weights)

df_grouped_sorted = df_grouped.sort_values(by="eco_score", ascending=False)

df_grouped_sorted.head(10)

,diet_group,sex,age_group,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,mean_acid,n_participants,mean_ghgs_norm,mean_ghgs_ch4_norm,mean_ghgs_n2o_norm,mean_acid_norm,mean_eut_norm,eco_score
48,vegan,female,20-29,2.383216,4.049673,13526.411131,10.647845,0.138213,0.198466,109.763872,411.915658,10.489061,487,1.000000,0.988486,1.000000,1.000000,1.000000,0.997348
58,vegan,male,60-69,2.427011,4.559576,17095.889356,10.762748,0.108715,0.213272,117.694371,477.762362,11.452591,52,0.995568,1.000000,0.979403,0.976875,0.997078,0.989979
57,vegan,male,50-59,2.524737,4.413799,14370.528918,11.197674,0.142867,0.211574,122.925271,416.335442,11.196164,62,0.985677,0.986669,0.981765,0.983030,0.986020,0.984670
49,vegan,female,30-39,2.540393,4.262573,13977.927844,11.121256,0.151392,0.212639,123.441892,425.503600,11.007468,376,0.984093,0.983341,0.980282,0.987558,0.987963,0.984423
51,vegan,female,50-59,2.520497,4.427549,15130.223558,10.830692,0.142114,0.218735,125.142393,424.692831,11.467557,106,0.986106,0.986963,0.971802,0.976516,0.995351,0.982986
59,vegan,male,70-79,2.587987,4.499390,16181.930818,11.433533,0.164137,0.217563,119.285600,460.111697,11.543243,19,0.979276,0.978366,0.973433,0.974700,0.980023,0.977134
52,vegan,female,60-69,2.633645,4.742450,15675.198731,11.333484,0.145497,0.223279,128.762133,439.410938,11.907675,81,0.974655,0.985642,0.965481,0.965954,0.982566,0.974851
55,vegan,male,30-39,2.698172,4.490259,15193.403259,11.620286,0.145903,0.217411,126.109666,452.753288,11.770708,251,0.968124,0.985484,0.973644,0.969241,0.975274,0.974482
56,vegan,male,40-49,2.694366,4.553664,14841.207696,11.630236,0.147843,0.224856,132.012334,441.906189,11.678908,165,0.968510,0.984727,0.963287,0.971444,0.975021,0.972757
50,vegan,female,40-49,2.657602,4.525626,15304.059746,11.714303,0.172362,0.222044,127.590251,451.688275,11.872144,219,0.972230,0.975156,0.967199,0.966806,0.972883,0.970951


In [33]:
df_grouped_sorted["diet_group"] = df_grouped_sorted["diet_group"].replace({
    "meat50": "meat<50",
    "meat": "meat50-99"
})
df_grouped_sorted["diet_group"].unique()

array(['vegan', 'veggie', 'fish', 'meat<50', 'meat50-99', 'meat100'],
      dtype=object)

In [35]:
df_grouped_sorted.head()

,diet_group,sex,age_group,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,mean_acid,n_participants,mean_ghgs_norm,mean_ghgs_ch4_norm,mean_ghgs_n2o_norm,mean_acid_norm,mean_eut_norm,eco_score
48,vegan,female,20-29,2.383216,4.049673,13526.411131,10.647845,0.138213,0.198466,109.763872,411.915658,10.489061,487,1.000000,0.988486,1.000000,1.000000,1.000000,0.997348
58,vegan,male,60-69,2.427011,4.559576,17095.889356,10.762748,0.108715,0.213272,117.694371,477.762362,11.452591,52,0.995568,1.000000,0.979403,0.976875,0.997078,0.989979
57,vegan,male,50-59,2.524737,4.413799,14370.528918,11.197674,0.142867,0.211574,122.925271,416.335442,11.196164,62,0.985677,0.986669,0.981765,0.983030,0.986020,0.984670
49,vegan,female,30-39,2.540393,4.262573,13977.927844,11.121256,0.151392,0.212639,123.441892,425.503600,11.007468,376,0.984093,0.983341,0.980282,0.987558,0.987963,0.984423
51,vegan,female,50-59,2.520497,4.427549,15130.223558,10.830692,0.142114,0.218735,125.142393,424.692831,11.467557,106,0.986106,0.986963,0.971802,0.976516,0.995351,0.982986


In [37]:
df_grouped_sorted

,diet_group,sex,age_group,mean_ghgs,mean_land,mean_watscar,mean_eut,mean_ghgs_ch4,mean_ghgs_n2o,mean_bio,mean_watuse,mean_acid,n_participants,mean_ghgs_norm,mean_ghgs_ch4_norm,mean_ghgs_n2o_norm,mean_acid_norm,mean_eut_norm,eco_score
48,vegan,female,20-29,2.383216,4.049673,13526.411131,10.647845,0.138213,0.198466,109.763872,411.915658,10.489061,487,1.000000,0.988486,1.000000,1.000000,1.000000,0.997348
58,vegan,male,60-69,2.427011,4.559576,17095.889356,10.762748,0.108715,0.213272,117.694371,477.762362,11.452591,52,0.995568,1.000000,0.979403,0.976875,0.997078,0.989979
57,vegan,male,50-59,2.524737,4.413799,14370.528918,11.197674,0.142867,0.211574,122.925271,416.335442,11.196164,62,0.985677,0.986669,0.981765,0.983030,0.986020,0.984670
49,vegan,female,30-39,2.540393,4.262573,13977.927844,11.121256,0.151392,0.212639,123.441892,425.503600,11.007468,376,0.984093,0.983341,0.980282,0.987558,0.987963,0.984423
51,vegan,female,50-59,2.520497,4.427549,15130.223558,10.830692,0.142114,0.218735,125.142393,424.692831,11.467557,106,0.986106,0.986963,0.971802,0.976516,0.995351,0.982986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,meat100,male,40-49,11.493864,24.185439,21888.216900,43.077655,2.441364,0.840707,426.420537,932.619265,46.705031,616,0.077942,0.089450,0.106546,0.130825,0.175416,0.110981
33,meat100,male,50-59,11.678040,24.765494,22196.915969,42.540711,2.493825,0.845726,424.304588,948.052353,47.281691,591,0.059302,0.068972,0.099563,0.116985,0.189068,0.099923
29,meat100,female,70-79,11.990853,26.031131,22709.031906,41.205293,2.635514,0.858691,432.051822,980.562330,47.783484,274,0.027643,0.013664,0.081527,0.104942,0.223024,0.079040
35,meat100,male,70-79,12.263987,26.714116,22944.250125,43.001081,2.670518,0.888627,446.023655,982.878945,48.459355,168,0.000000,0.000000,0.039882,0.088722,0.177363,0.051438


In [39]:
df_grouped_sorted.to_csv("CW2_dataset_final.csv", index=False)